In [ ]:
! pip install --upgrade --quiet python-dotenv llama-index-llms-openai llama-index llama-index-embeddings-openai

In [1]:
from llama_index.core.indices.struct_store import SQLTableRetrieverQueryEngine
from llama_index.core import VectorStoreIndex, SQLDatabase, Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.query_engine import NLSQLTableQueryEngine

from llama_index.core.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)

from sqlalchemy import (
    create_engine,
    text
)

from dotenv import load_dotenv
load_dotenv()

True

### In this document
The first few cells go over the general SQL query engine provided by llamaindex. It will demonstrate how to get started via SQL Alchemy and the limitations of NLSQLTableQueryEngine.
You will see that because of context window limitations, you will either hit context window limits or the resulting query will not work because of model confusion due to the large amount of tables and/or columns.

### The dataset used
I chose [this database](https://www.kaggle.com/datasets/wyattowalsh/basketball) from Kaggle because it contained 16 tables with 682 columns. The dataset was Last Updated on Thurs, July 26th, 2023. Download the database and bring the .sqlite file into the directory.

### Reference Documentation
Im going through [this](https://docs.llamaindex.ai/en/stable/examples/index_structs/struct_indices/duckdb_sql_query/) documentation from Llamaindex for reference.

In [2]:
# Connecting to the sqlite db and testing the query output
engine = create_engine("sqlite:///nba.sqlite")
with engine.connect() as con:
    rows = con.execute(text("SELECT * from common_player_info limit 5"))
    for row in rows:
        print(row)

('76001', 'Alaa', 'Abdelnaby', 'Alaa Abdelnaby', 'Abdelnaby, Alaa', 'A. Abdelnaby', 'alaa-abdelnaby', '1968-06-24 00:00:00', 'Duke', 'USA', 'Duke/USA', '6-10', '240', 5.0, '30', 'Forward', 'Inactive', 'N', 1610612757, 'Trail Blazers', 'POR', 'blazers', 'Portland', 'HISTADD_alaa_abdelnaby', 1990.0, 1994.0, 'N', 'Y', 'Y', '1990', '1', '25', 'N')
('76002', 'Zaid', 'Abdul-Aziz', 'Zaid Abdul-Aziz', 'Abdul-Aziz, Zaid', 'Z. Abdul-Aziz', 'zaid-abdul-aziz', '1946-04-07 00:00:00', 'Iowa State', 'USA', 'Iowa State/USA', '6-9', '235', 10.0, '54', 'Center', 'Inactive', 'N', 1610612745, 'Rockets', 'HOU', 'rockets', 'Houston', 'HISTADD_zaid_abdul-aziz', 1968.0, 1977.0, 'N', 'Y', 'Y', '1968', '1', '5', 'N')
('76003', 'Kareem', 'Abdul-Jabbar', 'Kareem Abdul-Jabbar', 'Abdul-Jabbar, Kareem', 'K. Abdul-Jabbar', 'kareem-abdul-jabbar', '1947-04-16 00:00:00', 'UCLA', 'USA', 'UCLA/USA', '7-2', '225', 20.0, '33', 'Center', 'Inactive', 'N', 1610612747, 'Lakers', 'LAL', 'lakers', 'Los Angeles', 'HISTADD_kareem_a

In [3]:
# Getting all the table names to populate the vector index
all_table_names = []
with engine.connect() as con:
    rows = con.execute(text("SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';"))
    for row in rows:
       all_table_names.append(row[0])
all_table_names

['game',
 'game_summary',
 'other_stats',
 'officials',
 'inactive_players',
 'game_info',
 'line_score',
 'play_by_play',
 'player',
 'team',
 'common_player_info',
 'team_details',
 'team_history',
 'draft_combine_stats',
 'draft_history',
 'team_info_common']

In [4]:
# Initializing the LLM. Make sure you have OPENAI_API_KEY in your .env file
llm = OpenAI(model_name="gpt-4-turbo",
             temperature=0,
             streaming=True)

embedding = OpenAIEmbedding()
Settings.llm = llm
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")
Settings.num_output = 500

In [5]:
# Setting up SQL Database using the SQL Alchemy engine above.
sql_database = SQLDatabase(engine)
query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database, llm=llm 
)

In [30]:
# This is the query that we will use 
query_str = "List the names of players who have played for more than three teams."
simple_sql_response = query_engine.query(query_str)

In [31]:
print(f"""
AI Response
{simple_sql_response.response}

Query Output
----------------------
{simple_sql_response.source_nodes[0].text}
      
Attempted Query
----------------------
{simple_sql_response.metadata['sql_query']}

""")


AI Response
The list of players who have played for more than three teams includes Quincy Acy, Arron Afflalo, Alexis Ajinca, Malik Allen, Tony Allen, Lou Amundson, Chris Andersen, Derek Anderson, Justin Anderson, Ryan Anderson, Joel Anthony, Carmelo Anthony, Trevor Ariza, Hilton Armstrong, D.J. Augustin, Luke Babbitt, Marcus Banks, Leandro Barbosa, Matt Barnes, Will Barton, Brandon Bass, Tony Battie, Jerryd Bayless, Aron Baynes, Kent Bazemore, Michael Beasley, Marco Belinelli, Raja Bell, DeAndre' Bembry, Anthony Bennett, Patrick Beverley, Chauncey Billups, Steve Blake, Eric Bledsoe, Keith Bogans, Bojan Bogdanovic, Andrew Bogut, Bol Bol, Trevor Booker, Calvin Booth, Avery Bradley, Elton Brand, Corey Brewer, Primoz Brezec, Aaron Brooks, MarShon Brooks, Moses Brown, Shannon Brown, Kwame Brown, Reggie Bullock, Trey Burke, Alec Burks, Jimmy Butler, Caron Butler, Andrew Bynum, Bruno Caboclo, Jose Calderon, Kentavious Caldwell-Pope, Marcus Camby, Isaiah Canaan, Rodney Carney, DeMarre Carroll

In [32]:
table_node_mapping = SQLTableNodeMapping(sql_database)

table_schema_objs = []
for table_name in all_table_names:
    table_schema_objs.append(SQLTableSchema(table_name=table_name))

obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex
)

In [33]:
obj_index.persist()

/var/folders/2n/s2mdnb396mb_p21bswvlnxb40000gn/T/ipykernel_7436/3545701165.py:1: UserWarning: Unable to persist ObjectNodeMapping. You will need to reconstruct the same object node mapping to build this ObjectIndex
  obj_index.persist()


In [34]:
reloaded_object_index = ObjectIndex.from_persist_dir(
    object_node_mapping=table_node_mapping  # without this, an error will be thrown
)

In [35]:
query_engine = SQLTableRetrieverQueryEngine(
    sql_database,
    reloaded_object_index.as_retriever(similarity_top_k=1),
)

In [36]:
response = query_engine.query(query_str)

In [37]:
print(f"""
AI Response
{response.response}

Query Output
----------------------
{response.source_nodes[0].text}
      
Attempted Query
----------------------
{response.metadata['sql_query']}

""")


AI Response
Some of the players who have played for more than three teams include Quincy Acy, Arron Afflalo, Alexis Ajinca, Tony Allen, Carmelo Anthony, Trevor Ariza, Lou Amundson, Chris Andersen, and many others.

Query Output
----------------------
[('Quincy', 'Acy'), ('Arron', 'Afflalo'), ('Alexis', 'Ajinca'), ('Malik', 'Allen'), ('Tony', 'Allen'), ('Lou', 'Amundson'), ('Chris', 'Andersen'), ('Derek', 'Anderson'), ('Justin', 'Anderson'), ('Ryan', 'Anderson'), ('Joel', 'Anthony'), ('Carmelo', 'Anthony'), ('Trevor', 'Ariza'), ('Hilton', 'Armstrong'), ('D.J.', 'Augustin'), ('Luke', 'Babbitt'), ('Marcus', 'Banks'), ('Leandro', 'Barbosa'), ('Matt', 'Barnes'), ('Will', 'Barton'), ('Brandon', 'Bass'), ('Tony', 'Battie'), ('Jerryd', 'Bayless'), ('Aron', 'Baynes'), ('Kent', 'Bazemore'), ('Michael', 'Beasley'), ('Marco', 'Belinelli'), ('Raja', 'Bell'), ("DeAndre'", 'Bembry'), ('Anthony', 'Bennett'), ('Patrick', 'Beverley'), ('Chauncey', 'Billups'), ('Steve', 'Blake'), ('Eric', 'Bledsoe'), ('